## Project: Actual Implementation of the Tree Using OOPS
##### BY: ADITI DONA

In [46]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from math import log2

In [47]:
iris = datasets.load_iris()

In [48]:
df = pd.DataFrame(iris.data)
df.columns = ["sl", "sw", 'pl', 'pw']

In [49]:
#Function to find label for a value
#if MIN_Value <=val < (m + Mean_Value) / 2 then it is assigned label a
#if (m + Mean_Value) <=val < Mean_Value then it is assigned label b
#if (Mean_Value) <=val < (Mean_Value + MAX_Value)/2 then it is assigned label c
#if (Mean_Value + MAX_Value)/2 <=val <= MAX_Value  then it is assigned label d

def label(val, *boundaries):
    if (val < boundaries[0]):
        return 'a'
    elif (val < boundaries[1]):
        return 'b'
    elif (val < boundaries[2]):
        return 'c'
    else:
        return 'd'

#Function to convert a continuous data into labelled data
#There are 4 lables  - a, b, c, d
def toLabel(df, old_feature_name):
    second = df[old_feature_name].mean()
    minimum = df[old_feature_name].min()
    first = (minimum + second)/2
    maximum = df[old_feature_name].max()
    third = (maximum + second)/2
    return df[old_feature_name].apply(label, args= (first, second, third))

In [50]:
#Convert all columns to labelled data
df['sl_labeled'] = toLabel(df, 'sl')
df['sw_labeled'] = toLabel(df, 'sw')
df['pl_labeled'] = toLabel(df, 'pl')
df['pw_labeled'] = toLabel(df, 'pw')
df

,sl,sw,pl,pw,sl_labeled,sw_labeled,pl_labeled,pw_labeled
0,5.1,3.5,1.4,0.2,b,c,a,a
1,4.9,3.0,1.4,0.2,a,b,a,a
2,4.7,3.2,1.3,0.2,a,c,a,a
3,4.6,3.1,1.5,0.2,a,c,a,a
4,5.0,3.6,1.4,0.2,a,c,a,a
...,...,...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,c,b,c,d
146,6.3,2.5,5.0,1.9,c,a,c,d
147,6.5,3.0,5.2,2.0,c,b,c,d
148,6.2,3.4,5.4,2.3,c,c,d,d


In [51]:
df.drop(['sl', 'sw', 'pl', 'pw'], axis = 1, inplace = True)

In [52]:
df.columns=['sapleLength','sapleWidth','petalLength','petalWidth']

In [53]:
df.head(10)

,sapleLength,sapleWidth,petalLength,petalWidth
0,b,c,a,a
1,a,b,a,a
2,a,c,a,a
3,a,c,a,a
4,a,c,a,a
5,b,d,a,a
6,a,c,a,a
7,a,c,a,a
8,a,b,a,a
9,a,c,a,a


In [54]:
y=iris.target
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

## Entropy

In [55]:
def entropy(y):
    
# Total entries
    total=len(y)
    
# Set of distinct values(classes)
    classes=set(y)
    
# Entropy calculation
    classCount=0
    entropy=0.0
    for i in classes:
        classCount=len(y[y==i])
        entropy+=-(classCount/total)*log2(classCount/total)
        
# Entropy of the Current Node    
    return entropy

## Gain Ratio

In [56]:
def gainRatio(df,y,selectedFeature):
    
# initial entropy
    initial_entropy=entropy(y)

# calculating entropy after splitting and split info
    total=df.count()[0]
    final_entropy=0
    split_info=0
    classes=set(df[selectedFeature])
    for i in classes:
        classCount=len(df[df[selectedFeature]==i])
        new_df=df[df[selectedFeature]==i]
        final_entropy+=(classCount/total)*entropy(new_df['target'])
        split_info-=(classCount/total)*log2(classCount/total)        
        
    
# information gain
    info_gain=initial_entropy-final_entropy
    gain_ratio=info_gain/split_info
    
    return gain_ratio

## Predict 

##### Function to predict the class based on current majority

In [57]:
def predict(y): 
    classes=set(y)
    maxCount=0
    prediction=""
    for i in classes:
        classCount=len(y[y==i])
        if classCount>maxCount:
            maxCount=classCount
            prediction=i
            
# return prediction
    return prediction

## TreeNode class

In [58]:
class TreeNode:
    def __init__(self,y,feature_to_split,gain_ratio,level):

        # Parameters :
        
            # level : current level
                self.level=level
            
            # y : Current output list of the chosen feature
                self.y=y
                
            # Sample : Number of samples in this split
                self.samples=len(y)
                
            # Value: List of number of samples in each class
                self.value=[]
                for i in sorted(set(y)):
                    classCount=len(y[y==i])
                    self.value.append(classCount)
                
            # feature_to_split : feature upon which the node is to be split and it is None for leaf node
                self.feature_to_split = feature_to_split
                
            # current_output   : Class with current majority at this instance to be predicted
                self.current_output = predict(y)
                
            # gain_ratio : current gain_ratio
                self.gain_ratio=gain_ratio
                
            # children : List of children
                self.children = []
                    
    def add_child(self,child):
        # Function : 
            # add_child() : Adds a child node to the Tree
        # Parameters :
            # child        : Reference of child node   
        self.children.append(child) 
        
    def getNodePrediction(self):
        # Function: to return current prediction
        d={0:"setosa",1:"versicolor",2:"verginica"}
        return d[self.current_output]

## Build Actual Tree Function

In [59]:
def build_actual_tree(root,df,y,unused_features):
# base case
# y contains only one distinct value-----> Pure Node is reached
    if len(set(root.y))==1:
        return    
# Unused_features is empty-----> No more splitting possible
    if(len(unused_features)==0):
        return
    
# Calculation of the best features to split upon using the GAIN_RATIO given by the UNUSED_FEATURES
    best_feature=""          # To store best_feature
    maxGainRatio=0           # To store the Max gain Ratio 
    for i in unused_features:
        feature_gain=gainRatio(df,y,i)          # Funtion returns the Gain Ratio for a Selected Feature
        if(feature_gain>maxGainRatio):
            maxGainRatio=feature_gain
            best_feature=i
    
# Updating the current node with the BEST_FEATURE and the GAIN_RATIO 
    root.feature_to_split=best_feature
    root.gain_ratio=maxGainRatio
    
# Updating the unused_features list by removing the selected feature
    unused_features.remove(best_feature)
    
# Splitting the ROOT into its children based on the number of disctinct values in the selected_feature column
    classes=set(df[best_feature])

# Splitting the ROOT
    for i in classes:
        new_df=df[df[best_feature]==i]
        new_y=new_df['target']
        
    # Initially, feature_to_split=None, gain_ratio=0.0
        child=TreeNode(new_y,None,0.0,root.level+1)  
        root.add_child(child)
        build_actual_tree(child,new_df,new_y,unused_features)

## printTree Function

In [60]:
def printTree(root):
    
# base acse handled by the for loop
    
    print('Level :',root.level)
    print('Split on feature :',root.feature_to_split)
    print('Gain Ratio :',root.gain_ratio)
    print('Samples :',root.samples)
    print('Value :',root.value)
    print('Class =',root.getNodePrediction())
    print()

    for i in root.children:
        printTree(i)

In [63]:
def main(df,y):
    df['target']=y   # df contains both feature column and target column together

    unused_features = set(df.columns) # list of features which will we used to split
    unused_features.remove('target') #removing the target column from the unused_featured

    # Initially 
    feature_to_split=None 
    level=0
    gain_ratio=0.0
    
    # Creating a ROOT node
    root=TreeNode(y,feature_to_split,gain_ratio,level)
    
    # Function call to build the tree
    build_actual_tree(root,df,y,unused_features)
    
    #Function call to print tree
    printTree(root)

In [64]:
main(df,y)

Level : 0
Split on feature : petalWidth
Gain Ratio : 0.6996382036222091
Samples : 150
Value : [50, 50, 50]
Class = setosa

Level : 1
Split on feature : petalLength
Gain Ratio : 0.4334099495621067
Samples : 56
Value : [40, 16]
Class = versicolor

Level : 2
Split on feature : sapleLength
Gain Ratio : 0.12674503775809332
Samples : 47
Value : [39, 8]
Class = versicolor

Level : 3
Split on feature : None
Gain Ratio : 0.0
Samples : 14
Value : [14]
Class = versicolor

Level : 3
Split on feature : None
Gain Ratio : 0.0
Samples : 1
Value : [1]
Class = verginica

Level : 3
Split on feature : sapleWidth
Gain Ratio : 0.07092036405148876
Samples : 30
Value : [23, 7]
Class = versicolor

Level : 4
Split on feature : None
Gain Ratio : 0.0
Samples : 4
Value : [3, 1]
Class = versicolor

Level : 4
Split on feature : None
Gain Ratio : 0.0
Samples : 6
Value : [6]
Class = versicolor

Level : 4
Split on feature : None
Gain Ratio : 0.0
Samples : 20
Value : [14, 6]
Class = versicolor

Level : 3
Split on featur